#### Importation des librairies :

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

### Chargement des datasets

In [2]:
# Chemin vers le dossier contenant les fichiers extraits
dossier_data = "C:/Users/paulm/Documents/Projet 7/Projet7withCSV/data"

In [10]:
# Fonction pour charger les données
def load_data(file_name):
    return pd.read_csv(os.path.join(dossier_data, file_name))

# Chargement des données d'application_train
df_test = load_data('processed_data_test.csv')

### Suppression des colonnes non pertinentes

In [13]:
# Suppression des colonnes avec trop de valeurs uniques
colonnes_a_supprimer = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']
df_test.drop(columns=colonnes_a_supprimer, inplace=True)

In [16]:
# Sélectionner uniquement les colonnes numériques
colonnes_numeriques = df.select_dtypes(include=[np.number])

# Identifier les colonnes contenant des valeurs infinies
colonnes_infinies = colonnes_numeriques.columns.to_series()[np.isinf(colonnes_numeriques).any()]

# Identifier les lignes contenant des valeurs infinies
lignes_avec_infinis = df.index[np.isinf(colonnes_numeriques).any(axis=1)]

# Suppression des lignes contenant des valeurs infinies
df.drop(lignes_avec_infinis, inplace=True)

# Sélection des colonnes numériques après la suppression des lignes
colonnes_numeriques = df.select_dtypes(include=[np.number])

# Vérification
print("\nReste-t-il des valeurs infinies ?")
reste_infinis = np.isinf(colonnes_numeriques).any().any()
print(reste_infinis)

# Si des valeurs infinies restent, les remplacer par NaN
if reste_infinis:
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("\nLes valeurs infinies ont été remplacées par NaN.")

# Vérification finale
reste_infinis = np.isinf(colonnes_numeriques).any().any()
print("\nReste-t-il des valeurs infinies après remplacement ?")
print(reste_infinis)

# Conserver la colonne d'identifiant unique
id_column = 'SK_ID_CURR'
df_id = df[[id_column]]  # Conserver les identifiants

# Encodage des variables catégorielles avec OrdinalEncoder (hors colonne d'identifiant unique)
ordinal_encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df.drop(columns=[id_column])), columns=df.drop(columns=[id_column]).columns)

# Suppression des colonnes non numériques
non_numeric_cols = df_encoded.select_dtypes(exclude=[np.number]).columns
df_encoded.drop(columns=non_numeric_cols, inplace=True)

# Gestion des valeurs manquantes avec SimpleImputer
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(df_encoded), columns=df_encoded.columns)

# Normalisation des données
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Enregistrement des données prétraitées
X_to_save = X.copy()
X_to_save['SK_ID_CURR'] = df_id['SK_ID_CURR'].values

# Utilisation de la méthode reindex pour réorganiser les colonnes
columns = ['SK_ID_CURR'] + [col for col in X_to_save.columns if col != 'SK_ID_CURR']
X_to_save = X_to_save[columns]

# Enregistrez le DataFrame dans un fichier CSV
X_to_save.to_csv('X_test_predictionV2.csv', index=False)


Reste-t-il des valeurs infinies ?
False

Reste-t-il des valeurs infinies après remplacement ?
False


In [3]:
# Chemins vers les fichiers
chemin_X_prediction = 'C:/Users/paulm/Documents/Projet 7/Projet7withCSV/data/X_predictionV1.csv'
chemin_X_test = 'C:/Users/paulm/Documents/Projet 7/Projet7withCSV/model_drift_analysis/X_test_predictionV2.csv'
chemin_X_test_final = 'C:/Users/paulm/Documents/Projet 7/Projet7withCSV/model_drift_analysis/X_test_prediction_encoded.csv'

# Charger le fichier d'entraînement pour obtenir l'ordre des colonnes
df_train = pd.read_csv(chemin_X_prediction)
colonnes_attendues = df_train.columns.tolist()

# Charger les données de test
test_data = pd.read_csv(chemin_X_test)

# Ajouter les colonnes manquantes avec des valeurs NaN
colonnes_manquantes = [col for col in colonnes_attendues if col not in test_data.columns]
for col in colonnes_manquantes:
    test_data[col] = pd.NA

# Réorganiser les colonnes du DataFrame de test pour qu'elles correspondent à l'ordre du fichier d'entraînement
test_data = test_data[colonnes_attendues]

# Encoder les colonnes catégorielles
categorical_columns = test_data.select_dtypes(include=['category', 'object']).columns
ordinal_encoder = OrdinalEncoder()
test_data[categorical_columns] = ordinal_encoder.fit_transform(test_data[categorical_columns])

# Convertir les colonnes en valeurs numériques
test_data = test_data.apply(pd.to_numeric, errors='coerce')

# Enregistrer le DataFrame de test prétraité dans un fichier CSV
test_data.to_csv(chemin_X_test_final, index=False)

# Vérifiez les modifications
print(test_data.head())
print(test_data.columns)


   SK_ID_CURR  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0      100001                 0.0          0.0           0.0              0.0   
1      100005                 0.0          0.0           0.0              0.0   
2      100013                 0.0          0.0           0.0              0.0   
3      100028                 0.0          0.0           0.0              0.0   
4      100038                 0.0          0.0           0.0              0.0   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0     -0.563843         -0.388937    0.342238    -0.579087         0.165654   
1     -0.563843         -1.124703   -0.953771    -0.858210        -1.031148   
2     -0.563843          0.662157    0.656255     1.971374         0.754245   
3      2.278347          1.389838    2.285562     1.394484         2.467698   
4      0.857252          0.371085    0.529546     0.360557         0.747705   

   ...  CC_CNT_INSTALMENT_MATURE_CUM_M